
<div class="alert alert-info" role="alert">
  <p>
    <center><b>Usage Guidelines</b></center>
  </p>

  <p>
    This lesson is part of the <b>DS Lab core curriculum</b>. For that reason, this notebook can only be used on your WQU virtual machine.
  </p>

  <p>
    This means:
    <ul>
      <li><span style="color: red">ⓧ</span> No downloading this notebook.</li>
      <li><span style="color: red">ⓧ</span> No re-sharing of this notebook with friends or colleagues.</li>
      <li><span style="color: red">ⓧ</span> No downloading the embedded videos in this notebook.</li>
      <li><span style="color: red">ⓧ</span> No re-sharing embedded videos with friends or colleagues.</li>
      <li><span style="color: red">ⓧ</span> No adding this notebook to public or private repositories.</li>
      <li><span style="color: red">ⓧ</span> No uploading this notebook (or screenshots of it) to other websites, including websites for study resources.</li>
    </ul>

  </p>
</div>


<font size="+3"><strong>8.3. Predicting Volatility</strong></font>

In the last lesson, we learned that one characteristic of stocks that's important to investors is **volatility**. Actually, it's so important that there are several time series models for predicting it. In this lesson, we'll build one such model called **GARCH**. We'll also continue working with assert statements to test our code. 

In [ ]:
import sqlite3

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wqet_grader
from arch import arch_model
from config import settings
from data import SQLRepository
from IPython.display import VimeoVideo
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

wqet_grader.init("Project 8 Assessment")

In [ ]:
VimeoVideo("770039650", h="c39b4b0c08", width=600)

# Prepare Data

As always, the first thing we need to do is connect to our data source.

## Import 

In [ ]:
VimeoVideo("770039537", h="a20af766cc", width=600)

**Task 8.3.1:** Create a connection to your database and then instantiate a `SQLRepository` named `repo` to interact with that database.

- [Open a connection to a SQL database using sqlite3.](../%40textbook/10-databases-sql.ipynb#sqlite3)

In [ ]:
connection = ...
repo = ...

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

Now that we're connected to a database, let's pull out what we need.

In [ ]:
VimeoVideo("770039513", h="74530cf5b8", width=600)

**Task 8.3.2:** Pull the most recent 2,500 rows of data for Ambuja Cement from your database. Assign the results to the variable `df_ambuja`.

- [Inspect a DataFrame using shape, info, and head in pandas.](../%40textbook/03-pandas-getting-started.ipynb#Inspecting-DataFrames)

In [ ]:
df_ambuja = ...

print("df_ambuja type:", type(df_ambuja))
print("df_ambuja shape:", df_ambuja.shape)
df_ambuja.head()

To train our model, the only data we need are the daily returns for `"AMBUJACEM.BSE"`. We learned how to calculate returns in the last lesson, but now let's formalize that process with a wrangle function.

In [ ]:
VimeoVideo("770039434", h="4fdcd5ffcb", width=600)

**Task 8.3.3:** Create a `wrangle_data` function whose output is the returns for a stock stored in your database. Use the docstring as a guide and the assert statements in the following code block to test your function. 

- [What's a function?](../%40textbook/02-python-advanced.ipynb#Functions)
- [Write a function in Python.](../%40textbook/02-python-advanced.ipynb#Functions)

In [ ]:
def wrangle_data():

    """Extract table data from database. Calculate returns.

    Parameters
    ----------
    ticker : str
        The ticker symbol of the stock (also table name in database).

    n_observations : int
        Number of observations to return.

    Returns
    -------
    pd.Series
        Name will be `"return"`. There will be no `NaN` values.
    """
    # Get table from database


    # Sort DataFrame ascending by date


    # Create "return" column


    # Return returns
    return ...

When you run the cell below to test your function, you'll also create a Series `y_ambuja` that we'll use to train our model.

In [ ]:
y_ambuja = wrangle_data(ticker="AMBUJACEM.BSE", n_observations=2500)

# Is `y_ambuja` a Series?
assert isinstance(y_ambuja, pd.Series)

# Are there 2500 observations in the Series?
assert len(y_ambuja) == 2500

# Is `y_ambuja` name "return"?
assert y_ambuja.name == "return"

# Does `y_ambuja` have a DatetimeIndex?
assert isinstance(y_ambuja.index, pd.DatetimeIndex)

# Is index sorted ascending?
assert all(y_ambuja.index == y_ambuja.sort_index(ascending=True).index)

# Are there no `NaN` values?
assert y_ambuja.isnull().sum() == 0

y_ambuja.head()

Great work! Now that we've got a wrangle function, let's get the returns for Suzlon Energy, too.

In [ ]:
VimeoVideo("770039414", h="8e8317029e", width=600)

**Task 8.3.4:** Use your `wrangle_data` function to get the returns for the 2,500 most recent trading days of Suzlon Energy. Assign the results to `y_suzlon`.

- [What's a function?](../%40textbook/02-python-advanced.ipynb#Functions)
- [Write a function in Python.](../%40textbook/02-python-advanced.ipynb#Functions)

In [ ]:
y_suzlon = ...

print("y_suzlon type:", type(y_suzlon))
print("y_suzlon shape:", y_suzlon.shape)
y_suzlon.head()

## Explore

Let's recreate the volatility time series plot we made in the last lesson so that we have a visual aid to talk about what volatility is.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot returns for `df_suzlon` and `df_ambuja`
y_suzlon.plot(ax=ax, label="SUZLON")
y_ambuja.plot(ax=ax, label="AMBUJACEM")

# Label axes
plt.xlabel("Date")
plt.ylabel("Return")

# Add legend
plt.legend();

The above plot shows how returns change over time. This may seem like a totally new concept, but if we visualize them without considering time, things will start to look familiar.

In [ ]:
VimeoVideo("770039370", h="dde163e45b", width=600)

**Task 8.3.5:** Create a histogram `y_ambuja` with 25 bins. Be sure to label the x-axis `"Returns"`, the y-axis `"Frequency [count]"`, and use the title `"Distribution of Ambuja Cement Daily Returns"`.

- [What's a histogram?](../%40textbook/08-visualization-plotly.ipynb#Histogram)
- [Create a histogram using Matplotlib.](../%40textbook/06-visualization-matplotlib.ipynb#Histograms)

In [ ]:
# Create histogram of `y_ambuja`, 25 bins


# Add axis labels



# Add title


This is a familiar shape! It turns out that returns follow an almost normal distribution, centered on `0`. **Volatility** is the measure of the spread of these returns around the mean. In other words, volatility in finance is the same thing at standard deviation in statistics.

Let's start by measuring the daily volatility of our two stocks. Since our data frequency is also daily, this will be exactly the same as calculating the standard deviation.

In [ ]:
VimeoVideo("770039332", h="d43d49b8e7", width=600)

**Task 8.3.6:** Calculate daily volatility for Suzlon and Ambuja, assigning them to the variables `suzlon_daily_volatility` and `ambuja_daily_volatility`, respectively.

- [What's volatility?](../%40textbook/05-pandas-summary-statistics.ipynb#Variance)
- [Calculate the volatility for an asset using Python.](../%40textbook/18-ts-models.ipynb#Calculating-Returns)

In [ ]:
suzlon_daily_volatility = ...
ambuja_daily_volatility = ...

print("Suzlon Daily Volatility:", suzlon_daily_volatility)
print("Ambuja Daily Volatility:", ambuja_daily_volatility)

Looks like Suzlon is more volatile than Ambuja. This reinforces what we saw in our time series plot, where Suzlon returns have a much wider spread.

While daily volatility is useful, investors are also interested in volatility over other time periods — like annual volatility. Keep in mind that a year isn't 365 days for a stock market, though. After excluding weekends and holidays, most markets have only 252 trading days.

So how do we go from daily to annual volatility? The same way we calculated the standard deviation for our multi-day experiment in Project 7!

In [ ]:
VimeoVideo("770039290", h="5b8452708a", width=600)

**Task 8.3.7:** Calculate the annual volatility for Suzlon and Ambuja, assigning the results to `suzlon_annual_volatility` and `ambuja_annual_volatility`, respectively.

- [What's volatility?](../%40textbook/05-pandas-summary-statistics.ipynb#Variance)
- [Calculate the volatility for an asset using Python.](../%40textbook/18-ts-models.ipynb#Calculating-Returns)

In [ ]:
suzlon_annual_volatility = ...
ambuja_annual_volatility = ...

print("Suzlon Annual Volatility:", suzlon_annual_volatility)
print("Ambuja Annual Volatility:", ambuja_annual_volatility)

Again, Suzlon has higher volatility than Ambuja. What do you think it means that the annual volatility is larger than daily?

Since we're dealing with time series data, another way to look at volatility is by calculating it using a rolling window. We'll do this the same way we calculated the rolling average for PM 2.5 levels in Project 3. Here, we'll start focusing on Ambuja Cement exclusively.

In [ ]:
VimeoVideo("770039248", h="71064ba910", width=600)

**Task 8.3.8:** Calculate the rolling volatility for `y_ambuja`, using a 50-day window. Assign the result to `ambuja_rolling_50d_volatility`.

- [What's a rolling window?](../%40textbook/17-ts-core.ipynb#Rolling-Averages)
- [Do a rolling window calculation in pandas.](../%40textbook/18-ts-models.ipynb#Making-a-Line-Plot-with-Rolling-Averages)

In [ ]:
ambuja_rolling_50d_volatility = ...

print("rolling_50d_volatility type:", type(ambuja_rolling_50d_volatility))
print("rolling_50d_volatility shape:", ambuja_rolling_50d_volatility.shape)
ambuja_rolling_50d_volatility.head()

This time, we'll focus on Ambuja Cement.

In [ ]:
VimeoVideo("770039209", h="8250d0a2d4", width=600)

**Task 8.3.9:** Create a time series plot showing the daily returns for Ambuja Cement and the 50-day rolling volatility. Be sure to label your axes and include a legend.

- [Make a line plot with time series data in pandas.](../%40textbook/18-ts-models.ipynb#Making-a-Line-Plot-with-Rolling-Averages)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot `y_ambuja`


# Plot `ambuja_rolling_50d_volatility`


# Add x-axis label


# Add legend


Here we can see that volatility goes up when the returns change drastically — either up or down. For instance, we can see a big increase in volatility in May 2020, when there were several days of large negative returns. We can also see volatility go down in August 2022, when there are only small day-to-day changes in returns.

This plot reveals a problem. We want to use returns to see if high volatility on one day is associated with high volatility on the following day. But high volatility is caused by large changes in returns, which can be either positive or negative. How can we assess negative and positive numbers together without them canceling each other out? One solution is to take the absolute value of the numbers, which is what we do to calculate performance metrics like mean absolute error. The other solution, which is more common in this context, is to square all the values.  

In [ ]:
VimeoVideo("770039182", h="1c7ee27846", width=600)

**Task 8.3.10:** Create a time series plot of the squared returns in `y_ambuja`. Don't forget to label your axes.

- [Make a line plot with time series data in pandas.](../%40textbook/07-visualization-pandas.ipynb#Line-Plots)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot squared returns


# Add axis labels



Perfect! Now it's much easier to see that (1) we have periods of high and low volatility, and (2) high volatility days tend to cluster together. This is a perfect situation to use a GARCH model.

A GARCH model is sort of like the ARMA model we learned about in Lesson 3.4. It has a `p` parameter handling correlations at prior time steps and a `q` parameter for dealing with "shock" events. It also uses the notion of lag. To see how many lags we should have in our model, we should create an ACF and PACF plot — but using the squared returns. 

In [ ]:
VimeoVideo("770039152", h="74c63d13ac", width=600)

**Task 8.3.11:** Create an ACF plot of squared returns for Ambuja Cement. Be sure to label your x-axis `"Lag [days]"` and your y-axis `"Correlation Coefficient"`.

- [What's an ACF plot?](../%40textbook/18-ts-models.ipynb#ACF-Plots)
- [Create an ACF plot using statsmodels.](../%40textbook/18-ts-models.ipynb#ACF-Plots)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Create ACF of squared returns


# Add axis labels



In [ ]:
VimeoVideo("770039126", h="4cfbc287d8", width=600)

**Task 8.3.12:** Create a PACF plot of squared returns for Ambuja Cement. Be sure to label your x-axis `"Lag [days]"` and your y-axis `"Correlation Coefficient"`.

- [What's a PACF plot?](../%40textbook/18-ts-models.ipynb#Creating-a-PACF-Plot)
- [Create a PACF plot using statsmodels.](../%40textbook/18-ts-models.ipynb#Creating-a-PACF-Plot)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Create PACF of squared returns


# Add axis labels



In our PACF, it looks like a lag of 3 would be a good starting point. 

Normally, at this point in the model building process, we would split our data into training and test sets, and then set a baseline. Not this time. This is because our model's input and its output are two different measurements. We'll use **returns** to train our model, but we want it to predict **volatility**. If we created a test set, it wouldn't give us the "true values" that we'd need to assess our model's performance. So this time, we'll skip right to iterating. 

## Split

The last thing we need to do before building our model is to create a training set. Note that we won't create a test set here. Rather, we'll use all of `y_ambuja` to conduct walk-forward validation after we've built our model. 

In [ ]:
VimeoVideo("770039107", h="8c9fbe0f4d", width=600)

**Task 8.3.13:** Create a training set `y_ambuja_train` that contains the first 80% of the observations in `y_ambuja`. 

In [ ]:
cutoff_test = ...
y_ambuja_train = ...

print("y_ambuja_train type:", type(y_ambuja_train))
print("y_ambuja_train shape:", y_ambuja_train.shape)
y_ambuja_train.tail()

# Build Model

Just like we did the last time we built a model like this, we'll begin by iterating.<span style='color: transparent; font-size:1%'>WQU WorldQuant University Applied Data Science Lab QQQQ</span>

## Iterate

In [ ]:
VimeoVideo("770039693", h="f06bf81928", width=600)

In [ ]:
VimeoVideo("770039053", h="beaf7753d4", width=600)

**Task 8.3.14:** Build and fit a GARCH model using the data in `y_ambuja`. Start with `3` as the value for `p` and `q`. Then use the model summary to assess its performance and try other lags.

- [What's a GARCH model?](../%40textbook/18-ts-models.ipynb#ARCH-and-GARCH-Models)
- [What's AIC?](../%40textbook/18-ts-models.ipynb#Common-Metrics)
- [What's BIC?](../%40textbook/18-ts-models.ipynb#Common-Metrics)
- [Build a GARCH model using arch.](../%40textbook/18-ts-models.ipynb#ARCH-and-GARCH-Models)

In [ ]:
# Build and train model
model = ...
print("model type:", type(model))

# Show model summary


<div class="alert alert-info" role="alert">
    <p><b>Tip:</b> You access the AIC and BIC scores programmatically. Every <code>ARCHModelResult</code> has an <code>.aic</code> and a <code>.bic</code> attribute. Try it for yourself: enter <code>model.aic</code> or <code>model.bic</code></p>
</div>

Now that we've settled on a model, let's visualize its predictions, together with the Ambuja returns.

In [ ]:
VimeoVideo("770039014", h="5e41551d9f", width=600)

**Task 8.3.15:** Create a time series plot with the Ambuja returns and the conditional volatility for your `model`. Be sure to include axis labels and add a legend.

- [Make a line plot with time series data in pandas.](../%40textbook/07-visualization-pandas.ipynb#Line-Plots)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot `y_ambuja_train`


# Plot conditional volatility * 2


# Plot conditional volatility * -2


# Add axis labels



# Add legend


Visually, our model looks pretty good, but we should examine residuals, just to make sure. In the case of GARCH models, we need to look at the standardized residuals. 

In [ ]:
VimeoVideo("770038994", h="2a13ab49a7", width=600)

**Task 8.3.16:** Create a time series plot of the standardized residuals for your `model`. Be sure to include axis labels and a legend.

- [Make a line plot with time series data in pandas.](../%40textbook/07-visualization-pandas.ipynb#Line-Plots)
- [What are standardized residuals in a GARCH model?](../%40textbook/18-ts-models.ipynb#Standardized-Residuals)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot standardized residuals


# Add axis labels



# Add legend


These residuals look good: they have a consistent mean and spread over time. Let's check their normality using a histogram. 

In [ ]:
VimeoVideo("770038970", h="f76c8f6529", width=600)

**Task 8.3.17:** Create a histogram with 25 bins of the standardized residuals for your model. Be sure to label your axes and use a title. 

- [What's a histogram?](../%40textbook/08-visualization-plotly.ipynb#Histogram)
- [Create a histogram using Matplotlib.](../%40textbook/06-visualization-matplotlib.ipynb#Histograms)

In [ ]:
# Create histogram of standardized residuals, 25 bins


# Add axis labels



# Add title


Our last visualization will the ACF of standardized residuals. Just like we did with our first ACF, we'll need to square the values here, too. 

In [ ]:
VimeoVideo("770038952", h="c7a3cfe34f", width=600)

**Task 8.3.18:** Create an ACF plot of the square of your standardized residuals. Don't forget axis labels!

- [What's an ACF plot?](../%40textbook/18-ts-models.ipynb#ACF-Plots)
- [Create an ACF plot using statsmodels.](../%40textbook/18-ts-models.ipynb#ACF-Plots)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Create ACF of squared, standardized residuals


# Add axis labels



Excellent! Looks like this model is ready for a final evaluation.

## Evaluate

To evaluate our model, we'll do walk-forward validation. Before we do, let's take a look at how this model returns its predictions.

In [ ]:
VimeoVideo("770038921", h="f74869b8fc", width=600)

**Task 8.3.19:** Create a one-day forecast from your `model` and assign the result to the variable `one_day_forecast`. 

- [What's variance?](../%40textbook/05-pandas-summary-statistics.ipynb#Variance)
- [Generate a forecast for a model using arch.](../%40textbook/18-ts-models.ipynb#Forecasting)

In [ ]:
one_day_forecast = ...

print("one_day_forecast type:", type(one_day_forecast))
one_day_forecast

There are two things we need to keep in mind here. First, our `model` forecast shows the predicted **variance**, not the **standard deviation** / **volatility**. So we'll need to take the square root of the value. Second, the prediction is in the form of a DataFrame. It has a DatetimeIndex, and the date is the last day for which we have training data. The `"h.1"` column stands for "horizon 1", that is, our model's prediction for the following day. We'll have to keep all this in mind when we reformat this prediction to serve to the end user of our application.

In [ ]:
VimeoVideo("770038861", h="10efe8c445", width=600)

**Task 8.3.20:** Complete the code below to do walk-forward validation on your `model`. Then run the following code block to visualize the model's test predictions.

- [What's walk-forward validation?](../%40textbook/17-ts-core.ipynb#Walk-Forward-Validation)
- [Perform walk-forward validation for time series model.](../%40textbook/17-ts-core.ipynb#Walk-Forward-Validation)

In [ ]:
# Create empty list to hold predictions
predictions = []

# Calculate size of test data (20%)
test_size = int(len(y_ambuja) * 0.2)

# Walk forward
for i in range(test_size):
    # Create test data
    y_train = y_ambuja.iloc[: -(test_size - i)]

    # Train model
    model = ...

    # Generate next prediction (volatility, not variance)
    next_pred = ...

    # Append prediction to list
    predictions.append(next_pred)

# Create Series from predictions list
y_test_wfv = pd.Series(predictions, index=y_ambuja.tail(test_size).index)

print("y_test_wfv type:", type(y_test_wfv))
print("y_test_wfv shape:", y_test_wfv.shape)
y_test_wfv.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot returns for test data
y_ambuja.tail(test_size).plot(ax=ax, label="Ambuja Return")

# Plot volatility predictions * 2
(2 * y_test_wfv).plot(ax=ax, c="C1", label="2 SD Predicted Volatility")

# Plot volatility predictions * -2
(-2 * y_test_wfv).plot(ax=ax, c="C1")

# Label axes
plt.xlabel("Date")
plt.ylabel("Return")

# Add legend
plt.legend();

This looks pretty good. Our volatility predictions seem to follow the changes in returns over time. This is especially clear in the low-volatility period in the summer of 2022 and the high-volatility period in fall 2022.

One additional step we could do to evaluate how our `model` performs on the test data would be to plot the ACF of the standardized residuals for only the test set. But you can do that step on your own.

# Communicate Results

Normally in this section, we create visualizations for a human audience, but our goal for *this* project is to create an API for a *computer* audience. So we'll focus on transforming our model's predictions to JSON format, which is what we'll use to send predictions in our application. 

The first thing we need to do is create a DatetimeIndex for our predictions. Using labels like `"h.1"`, `"h.2"`, etc., won't work. But there are two things we need to keep in mind. First, we can't include dates that are weekends because no trading happens on those days. And we'll need to write our dates using strings that follow the [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601) standard. 

In [ ]:
VimeoVideo("770038804", h="8976257596", width=600)

**Task 8.3.21:** Below is a `prediction`, which contains a 5-day forecast from our `model`. Using it as a starting point, create a `prediction_index`. This should be a list with the following 5 dates written in ISO 8601 format.

- [Create a fixed frequency DatetimeIndex in pandas.](../%40textbook/04-pandas-advanced.ipynb#Date-Time-Indices)
- [Transform a Timestamp to ISO 8601 format in pandas.](../%40textbook/04-pandas-advanced.ipynb#Time-Stamps)

In [ ]:
# Generate 5-day volatility forecast
prediction = model.forecast(horizon=5, reindex=False).variance ** 0.5
print(prediction)

# Calculate forecast start date
start = ...

# Create date range
prediction_dates = ...

# Create prediction index labels, ISO 8601 format
prediction_index = ...

print("prediction_index type:", type(prediction_index))
print("prediction_index len:", len(prediction_index))
prediction_index[:3]

Now that we know how to create the index, let's create a function to combine the index and predictions, and then return a dictionary where each key is a date and each value is a predicted volatility. 

In [ ]:
VimeoVideo("770039565", h="d419d0a78d", width=600)

**Task 8.3.22:** Create a `clean_prediction` function. It should take a variance prediction DataFrame as input and return a dictionary where each key is a date in ISO 8601 format and each value is the predicted volatility. Use the docstring as a guide and the assert statements to test your function. When you're satisfied with the result, submit it to the grader.

- [What's a function?](../%40textbook/02-python-advanced.ipynb#Functions)
- [Write a function in Python.](../%40textbook/02-python-advanced.ipynb#Functions)

In [ ]:
def clean_prediction():

    """Reformat model prediction to JSON.

    Parameters
    ----------
    prediction : pd.DataFrame
        Variance from a `ARCHModelForecast`

    Returns
    -------
    dict
        Forecast of volatility. Each key is date in ISO 8601 format.
        Each value is predicted volatility.
    """
    # Calculate forecast start date


    # Create date range
    

    # Create prediction index labels, ISO 8601 format


    # Extract predictions from DataFrame, get square root


    # Combine `data` and `prediction_index` into Series


    # Return Series as dictionary
    return ...

In [ ]:
prediction = model.forecast(horizon=10, reindex=False).variance
prediction_formatted = clean_prediction(prediction)

# Is `prediction_formatted` a dictionary?
assert isinstance(prediction_formatted, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction_formatted.keys())

# Are values correct data type
assert all(isinstance(v, float) for v in prediction_formatted.values())

prediction_formatted

In [ ]:
wqet_grader.grade("Project 8 Assessment", "Task 8.3.21", prediction_formatted)

Great work! We now have several components for our application: classes for getting data from an API, classes for storing it in a database, and code for building our model and cleaning our predictions. The next step is creating a class for our model and paths for application — both of which we'll do in the next lesson.

---
Copyright 2023 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
